In [417]:
import tkinter as tk
import webbrowser
from tkinter import ttk
from tkinter import filedialog
from PIL import ImageTk, Image
import string
import io
import csv
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup

In [418]:
# global var
W = np.array([[-1.234352  ],
       [-1.0965089 ],
       [ 0.9574655 ],
       [ 0.07142846],
       [ 0.18221012],
       [-1.110761  ],
       [ 0.49294937],
       [-0.5562565 ],
       [-0.27289093],
       [-0.3349833 ],
       [ 0.2631448 ],
       [-0.32123747],
       [ 1.0309917 ],
       [ 0.5341499 ],
       [-1.1661073 ],
       [-0.22655593],
       [-0.3114538 ],
       [ 0.50718987],
       [-0.9014772 ],
       [ 0.533391  ],
       [-0.1640348 ],
       [ 0.3487134 ],
       [-0.2405224 ],
       [ 0.54387933],
       [-0.12337092]])
b = np.array([[-0.99959964]])

wordsBag = ['likelyhood', 'breach', 'trade', 'evidence', 'property', 'public', 'opponents', 'constitution', 'fiduciary', 'confusion', 'crimes', 'copyright', 'intellectual', 'misuse', 'death', 'marks', 'constitutional', 'unregistered', 'contract', 'drugs', 'proprietor', 'similar', 'penalty', 'company', 'criminal']

countVec = []


Ww = np.array([[ 0.8594918 ],
       [ 0.8459306 ],
       [ 1.6249211 ],
       [ 1.8483686 ],
       [-0.36843148],
       [ 1.751518  ]])
bb = np.array([[-3.5971665]])

scoreVec = []

In [419]:
def choose_file(event=None):
    f = filedialog.askopenfilename()
    fileName.set(f)
    fileNameShow.set(f[-9:])

In [420]:
def open_browser():
    if len(fileName.get()) > 0:
        webbrowser.open('file://' + os.path.realpath(fileName.get()))

In [421]:
def sigmoid(a):
    return 1/(1+np.exp(-a))

In [422]:
def read_html(fileName):
    html = io.open(fileName.get(), mode="r", encoding="utf-8")
    soup = BeautifulSoup(html, 'html.parser') 
    return soup

In [423]:
def read_stop_words(stopFileName):
    file = open(stopFileName, mode="r", encoding="utf-8")
    words = list(map(lambda x: x[0:-1], file.readlines()))
    return words

In [424]:
def class_contents(soup, className):
    lst = soup.find_all(class_=className)
    text = '\n'.join([''.join(l.findAll(text=True)) + ' ' for l in lst])
    return text

In [425]:
def clean_text(s, stopWords):
    puncs = string.punctuation.translate({ord('('): None, ord(')'): None}) + '’' + '“' + '”' + '\\'
    return list(map(lambda x: x.lower(), list(filter(lambda x:  4 < len(x) <= 16 and not any(p in x for p in puncs) and x not in stopWords and x.count('(') == x.count(')'), s.split(' ')))))

In [426]:
def word_counter_vectorizer(text, wordsBag):
    vecLength = len(wordsBag)
    vec = [None] * vecLength
    for idx, word in enumerate(wordsBag):
        vec[idx] = text.count(word)
    return vec

In [427]:
def classify_case():
    model = np.matmul(countVec, W) + b
    prediction = np.sign(model)
    global caseType
    if prediction >= 0:
        caseType.set('Trade Mark')
    else:
        caseType.set('None Trade Mark')

In [428]:
def create_data(className='txt-body', stopWords='./stopWords.txt'):
    global countVec
    global caseSum
    if len(fileName.get()) > 0:
        soup = read_html(fileName)
        text = class_contents(soup, className)
        caseSum.set('\n'.join(list(filter(lambda x: len(x) < 50, text.split('\n')))[:]))
        cleanText = clean_text(text, read_stop_words(stopWords))
        countVec = word_counter_vectorizer(cleanText, wordsBag)
        countVec = np.array(countVec)
        countVec.astype(np.float64)
        classify_case()
    else:
        caseType.set('Choose a case first')  

In [429]:
def similarity_score(t, score):
    def _record_score(v):
        score.set(v)
    s = tk.Scale(t, label='Score', from_=0, to=10, orient=tk.HORIZONTAL,
             length=300, showvalue=0, tickinterval=1, resolution=0.5, command=_record_score, bg = '#ECECEC', fg='#005b96',
            font=('Helvitica', 14))
    s.pack()
    l = tk.Label(t, 
        textvariable= score,
        bg ='#ECECEC',
        fg = '#d9534f',
        font=('Helvitica', 16)
    )
    l.pack() 

In [430]:
def new_window(): # new window definition
    def _explain(s):
        explaination.set(dic[s])
    explaination = tk.StringVar()
    dic = {'1': '1', '2':'2', '3':'3', '4':'4', '5':'5', '6':'6'}
    newWin = tk.Toplevel(window)
    newWin.title('Trade Mark Cases')
    newWin.geometry('400x900')
    newWin.configure(background='#b3cde0')
    path = "./graph.png"
    img = ImageTk.PhotoImage(Image.open(path))
    lp = tk.Label(newWin, image=img, bg='#b3cde0')
    lp.pack() 
    l1 = tk.Label(newWin, text="3 Types Trade Mark Cases", fg = '#011f4b',
    bg='#b3cde0', font=('MS Sans Serif', 22))
    l1.pack()
    l2 = tk.Label(newWin, text="1. Text Logo\n2. Image Logo\n3. Text + Image Logo", fg = '#011f4b',
    bg='#b3cde0', font=('Helvitica', 20))
    l2.pack()
    l3 = tk.Label(newWin, 
        text='6 criterion:',
        fg = '#d9534f',
        bg='#b3cde0',
        font=('Helvitica', 22))
    l3.pack()
    b1 = tk.Button(newWin, 
        text ="1. Marks Similarity", 
        command =_explain('2'), 
        font=('Helvitica', 18),
        width=15, height=2)
    b1.pack()
    l4 = tk.Label(newWin, 
        textvariable=explaination,
        fg = '#d9534f',
        bg='#b3cde0',
        font=('Helvitica', 22))
    l4.pack()    
    b2 = tk.Button(newWin, 
        text ="Predict the Win Rate", 
        command =new_tab, 
        font=('Helvitica', 18),
        width=15, height=2)
    b2.pack()
    newWin.mainloop()

In [431]:
def new_tab():
    def _predict_result():
        global scoreVec
        global Ww
        global bb
        global winProb
        global winOrLose
        scoreVec = [score1.get(), score2.get(), score3.get(), score4.get(), score5.get(), score6.get()]
        scoreVec = np.asarray(scoreVec)
        scoreVec.astype(np.float64)
        model = np.matmul(scoreVec, Ww) + bb
        prediction = sigmoid(model)
        winProb.set(str(prediction[0][0] * 100)[0:4] + '%')
        if prediction[0][0] >= 0.7:
            winOrLose.set('Win')
        else:
            winOrLose.set('Lose')
        l1 = tk.Label(tab6, text=str(winProb.get()), bg='#ECECEC', fg='#d9534f',font=('Helvitica', 20))
        l1.pack()
        l2 = tk.Label(tab6, text=winOrLose.get(), bg='#ECECEC', fg='#d9534f', font=('Helvitica', 20))
        l2.pack()
        
    newTab = tk.Toplevel(window)
    newTab.title('Predict the Trade Mark Cases')
    newTab.geometry('800x600')
    newTab.configure(background='#b3cde0')
    # tab1
    tabControl = ttk.Notebook(newTab)       
    tab1 = ttk.Frame(tabControl)
    l1 = tk.Label(tab1, text="Marks Similarity example...", anchor='w', bg ='#ECECEC', fg = '#011f4b', font=('Helvitica', 20))
    l1.pack(fill='both')
    similarity_score(tab1, score1)
    tabControl.add(tab1, text='Marks Similarity')
    # tab2
    tab2 = ttk.Frame(tabControl)
    l2 = tk.Label(tab2, text="Visual Similarity example...", anchor='w', bg ='#ECECEC', fg = '#011f4b', font=('Helvitica', 20))
    l2.pack(fill='both')
    similarity_score(tab2, score2)
    tabControl.add(tab2, text='Visual Similarity')
    # tab3
    tab3 = ttk.Frame(tabControl)
    l3 = tk.Label(tab3, text="Aural Similarity example...", anchor='w', bg ='#ECECEC', fg = '#011f4b', font=('Helvitica', 20))
    l3.pack(fill='both')
    similarity_score(tab3, score3)
    tabControl.add(tab3, text='Aural Similarity')
    # tab4
    tab4 = ttk.Frame(tabControl)
    l4 = tk.Label(tab4, text="Conceptual Similarity example...", anchor='w', bg ='#ECECEC', fg = '#011f4b', font=('Helvitica', 20))
    l4.pack(fill='both')
    similarity_score(tab4, score4)
    tabControl.add(tab4, text='Conceptual Similarity')
    # tab5
    tab5 = ttk.Frame(tabControl)
    l5 = tk.Label(tab5, text="G&S Similarity example...", anchor='w', bg ='#ECECEC', fg = '#011f4b', font=('Helvitica', 20))
    l5.pack(fill='both')
    similarity_score(tab5, score5)
    tabControl.add(tab5, text='G&S Similarity')
    # tab6
    tab6 = ttk.Frame(tabControl)
    l6 = tk.Label(tab6, text="Likelyhood of Confusion example...", anchor='w', bg ='#ECECEC', fg = '#011f4b', font=('Helvitica', 20))
    l6.pack(fill='both')
    similarity_score(tab6, score6)
    b = tk.Button(tab6, 
        text ="Predict the Win Rate", 
        command =_predict_result, 
        bg ='#ECECEC',
        font=('Helvitica', 16))
    b.pack()
    tabControl.add(tab6, text='Likelyhood of Confusion')
    tabControl.pack(expand=1, fill="both") 
    newTab.mainloop() 

In [432]:
window = tk.Tk()
window.title('LIT')
window.geometry('420x900')
window.configure(background='#b3cde0')
fileName = tk.StringVar()
fileNameShow = tk.StringVar()
caseType = tk.StringVar()
caseSum = tk.StringVar()
score1 = tk.DoubleVar()
score2 = tk.DoubleVar()
score3 = tk.DoubleVar()
score4 = tk.DoubleVar()
score5 = tk.DoubleVar()
score6 = tk.DoubleVar()
winProb = tk.StringVar()
winOrLose = tk.StringVar()

In [433]:
path = "./stadistics.png"
img = ImageTk.PhotoImage(Image.open(path))
lp = tk.Label(window, image=img, bg='#b3cde0')
lp.pack() 

In [434]:
l0 = tk.Label(window, 
    text= 'Trade Mark Cases',
    fg = '#011f4b',
    bg='#b3cde0', 
    font=('MS Sans Serif', 32), 
    width=20)
l0.pack() 

In [435]:
l1 = tk.Label(window, 
    textvariable= fileNameShow,
    fg = '#011f4b',
    bg='#b3cde0',  
    font=('Helvitica', 16), 
    width=50, height=2)
l1.pack() 

In [436]:
b1 = tk.Button(window, 
    text='Choose a Case',
    command=choose_file,
    font=('Helvitica', 20),
    height = 2)
b1.pack()

In [437]:
l2 = tk.Label(window, 
    textvariable=caseType,
    fg = '#d9534f',
    bg='#b3cde0',  
    font=('Helvitica', 20),
    width=24, height=2)
l2.pack()

In [438]:
b2 = tk.Button(window, 
    text='Classify the Case', 
    command=create_data,
    font=('Helvitica', 20),
    height = 2)
b2.pack()

In [439]:
l3 = tk.Label(window, 
    textvariable=caseSum,
    fg = '#011f4b',
    bg='#b3cde0',  
    font=('Helvitica', 14),
    width=200, height=10)
l3.pack()

In [440]:
b3 = tk.Button(window, 
    text ="More Trade Mark Cases", 
    command=new_window,
    font=('Helvitica', 20),
    width=20, height=2)
b3.pack()
# b3.grid(row=9, column=1, sticky=S, ipady=30)
# b3.grid(column=0, row=7, padx=20, pady=40)

In [441]:
b4 = tk.Button(window, 
    text ="Read case", 
    command=open_browser,
    font=('Helvitica', 20),
    width=20, height=2)
b4.pack()

In [442]:
window.mainloop()